# Des classes pour représenter les données

## Pour les prénoms

In [1]:
case class Prenom(sexe: String, prenom: String, annee: Int, codeDept: Int, nombre: Int)

Intitializing Scala interpreter ...

Spark Web UI available at http://ca5f389260da:4043
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1607645857750)
SparkSession available as 'spark'


defined class Prenom


In [2]:
val ariane = Prenom("2", "ARIANE", 2007, 91, 10)
val cassiopee = Prenom("2", "CASSIOPEE", 2009, 91, 3)
val helios = Prenom("1", "HELIOS", 2012, 91, 1)
println(s"$ariane, $cassiopee, $helios")

Prenom(2,ARIANE,2007,91,10), Prenom(2,CASSIOPEE,2009,91,3), Prenom(1,HELIOS,2012,91,1)


ariane: Prenom = Prenom(2,ARIANE,2007,91,10)
cassiopee: Prenom = Prenom(2,CASSIOPEE,2009,91,3)
helios: Prenom = Prenom(1,HELIOS,2012,91,1)


## Pour les départements

In [3]:
case class Dept(region: Int, codeDept: String, chefLieu: String, typeNom: Int, nom: String, nomEnrichi: String)

defined class Dept


In [4]:
val puyDeDome = Dept(84, "63", "63113", 2, "PUY-DE-DOME", "Puy-de-Dôme")
println(puyDeDome)

Dept(84,63,63113,2,PUY-DE-DOME,Puy-de-Dôme)


puyDeDome: Dept = Dept(84,63,63113,2,PUY-DE-DOME,Puy-de-Dôme)


# Chargement des données

In [5]:
// Pour les conversions implicites de RDDs vers DataFrames
val sparkRO = spark // bricolage pour que cela fonctionne dans le notebool (inutile sinon)
import sparkRO.implicits._

sparkRO: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@14c895f5
import sparkRO.implicits._


## Chargement des prénoms

In [6]:
val lignes = spark.sparkContext.textFile("prenoms_sample.txt")
val prenomsRDD = lignes.map(_.split('\t')).map(a => Prenom(a(0), a(1), a(2).toInt, a(3).toInt, a(4).toDouble.toInt))
val prenoms = prenomsRDD.toDS()
prenoms.show() // échoue ici mais fonctionne dans spark-shell ?!?

## Chargement des départements

In [7]:
val lignesDept = spark.sparkContext.textFile("dpts.txt").filter(l => l.startsWith("REGION") == false)
val deptsRDD = lignesDept.map(_.split('\t')).map(a => Dept(a(0).toInt, a(1), a(2), a(3).toInt, a(4), a(5)))
val depts = deptsRDD.toDS()
depts.show() // échoue ici mais fonctionne dans spark-shell ?!?

# Sauve les données au format parquet

## Les prénoms partitionnés par départements et années et compressés (Snappy)

In [10]:
prenoms.write.partitionBy("codeDept", "annee").for
mat("parquet").save("prenomsParDeptsEtAnnees.parquet")

<console>: 2: error: identifier expected but 'for' found.

## Les prénoms partitionnés par départements et années et compressés (gzip)

In [8]:
prenoms.write.partitionBy("codeDept", "annee").option("compression", "gzip").format("parquet").save("prenomsParDeptsEtAnnees.gzip.parquet")

<console>: 31: error: not found: value prenoms

## Les départements

In [9]:
depts.write.format("parquet").save("depts.parquet")

<console>: 31: error: not found: value depts